### Pre-Instalation

While the library is in development the conda build library will help make importing the library easier in projects.

In [ ]:
!sudo /opt/conda/bin/conda install conda-build -y

### Instalation

Conda Build can be used to add the in development mosaiQue library to conda for easy importing.

In [ ]:
!sudo /opt/conda/bin/conda-develop -n QML-QPF PATH /workspaces/QML-QPF/mosaiQue

### Imports

In [ ]:
import mosaique as mq
from concurrent.futures import ProcessPoolExecutor, as_completed
import itertools
import numpy as np
import pennylane as qml
import os
from mosaique.models.operation import OperationLayer

In [ ]:

# Set the environment for asynchronous GPU usage
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'


### Define the Quantum Circuit

Set up a 4 wire quantum node using PennyLane.

In [ ]:

def operation():
    dev = qml.device("default.qubit.tf", wires=4)
    @qml.qnode(dev, interface='tf')
    def cnot(inputs):
        inputs = inputs * np.pi
        qml.AngleEmbedding(inputs[:,...], wires=range(4), rotation='Y')

        qml.CNOT(wires=[0, 1])
        qml.CNOT(wires=[2, 3])

        # Measurement producing 4 classical output values
        return [qml.expval(qml.PauliZ(j)) for j in range(4)]
    return cnot


### Prepare the Dataset

Load the dataset and fit the convolution layers.

In [ ]:

def preset():
    from tensorflow import keras
    mnist_dataset = keras.datasets.mnist
    tr_layer = mq.ConvolutionLayer4x4("mnist_train5")
    te_layer = mq.ConvolutionLayer4x4("mnist_test5")
    (tr_images, tr_labels), (te_images, te_labels) = mnist_dataset.load_data()
    tr_layer.fit(tr_images)
    te_layer.fit(te_images)
    tr_images = tr_layer.transform(tr_images)
    te_images = te_layer.transform(te_images)
    return ((tr_layer, (tr_images, tr_labels)), (te_layer,(te_images, te_labels)))

with ProcessPoolExecutor() as executor:
    future = executor.submit(preset)

((train_layer, (train_images, train_labels)), (test_layer,(test_images, test_labels))) = future.result() #This blocks until the task completes


### Prepare Permutations and Processing Pool

Generate process pool and the permutations of the wire assignments.

In [ ]:

permutations = np.asarray(list(itertools.permutations(range(4))))

def pool(x, call, p, l):
    op = OperationLayer(call())
    predict = l.post_transform(op.pre_op.predict(x,batch_size=1000))
    l.save(predict, p)


### Process the Train Images in Parallel:

Utilize the ProcessPoolExecutor to process the train images concurrently.

In [ ]:

for j in range(3):
    with ProcessPoolExecutor(8) as executor:
        runner = {
            executor.submit(pool, x=train_images[:,:,p], call=operation, p=p, l=train_layer): p for p in permutations[8*j:8*(j+1)]
        }
        for future in as_completed(runner):
            runner.pop(future)
#3 min 43 sec


### Process the Test Images in Parallel:

Utilize the ProcessPoolExecutor to process the test images concurrently.

In [ ]:

for j in range(3):
    with ProcessPoolExecutor(8) as executor:
        runner = {
            executor.submit(pool, x=test_images[:,:,p], call=operation, p=p, l=test_layer): p for p in permutations[8*j:8*(j+1)]
        }
        for future in as_completed(runner):
            runner.pop(future)
#50 sec
